In [1]:
import requests
import time
from datetime import datetime, timedelta
from pytz import timezone
from copy import deepcopy
import pandas as pd

In [2]:
pollutants = ['PM10', 'PM25', 'O3', 'CO', 'NO2', 'SO2']
stations = ['Karpos', 'Centar', 'GaziBaba', 'Lisice', 'Rektorat', 'Miladinovci']

In [3]:
date_start = datetime(2000, 1, 1, 0, 0, 0)
date_end = datetime(2020, 4, 1, 0, 0, 0)
period = timedelta(days=150)
offset = timedelta(seconds=1)

In [4]:
sleep_time = 5

for station in stations:
    for pollutant in pollutants:
        results = []
        curr_start = deepcopy(date_start)
        while curr_start < date_end:
            try:
                time.sleep(5)
                
                curr_end = curr_start + period - offset
                print(curr_start.strftime("%d/%m/%Y, %H:%M:%S"), '-', curr_end.strftime("%d/%m/%Y, %H:%M:%S"), 
                     station, pollutant)

                URL = f'http://air.moepp.gov.mk/graphs/site/pages/MakeGraph.php?station={station}'+\
                      f'&parameter={pollutant}'+\
                      f'&beginDate={curr_start.strftime("%Y-%m-%d")}'+\
                      f'&beginTime={curr_start.strftime("%H:%M")}'+\
                      f'&endDate={curr_end.strftime("%Y-%m-%d")}'+\
                      f'&endTime={curr_end.strftime("%H:%M")}'+\
                      f'&i=1585697894082&lang=mk'

                curr_start += period

                r = requests.get(url = URL) 
                data = r.json()

                # sanity check to make sure the jason has all the expected fields 
                if 'measurements' not in data or len(data['measurements']) == 0 \
                    or 'data' not in data['measurements'][0]:
                    continue

                measurements = data['measurements'][0]['data']
                timestamps = data['times']

                rows = list(zip(timestamps, measurements))
                results.extend(rows)
                
                time_sleep = 5
            
            except Exception as e:
                print(e)
                time.sleep(sleep_time)
                sleep_time += 5
                
                  
        
        with open(f'data/raw-sensor/{station}_{pollutant}', 'w+') as f:
            f.write('Timestamp,Value\n')
            for timestamp, value in results:
                f.write(f'{timestamp},{value}\n')
    

01/01/2000, 00:00:00 - 29/05/2000, 23:59:59 Karpos PM10
30/05/2000, 00:00:00 - 26/10/2000, 23:59:59 Karpos PM10
27/10/2000, 00:00:00 - 25/03/2001, 23:59:59 Karpos PM10
26/03/2001, 00:00:00 - 22/08/2001, 23:59:59 Karpos PM10
23/08/2001, 00:00:00 - 19/01/2002, 23:59:59 Karpos PM10
20/01/2002, 00:00:00 - 18/06/2002, 23:59:59 Karpos PM10
19/06/2002, 00:00:00 - 15/11/2002, 23:59:59 Karpos PM10
16/11/2002, 00:00:00 - 14/04/2003, 23:59:59 Karpos PM10
15/04/2003, 00:00:00 - 11/09/2003, 23:59:59 Karpos PM10
12/09/2003, 00:00:00 - 08/02/2004, 23:59:59 Karpos PM10
09/02/2004, 00:00:00 - 07/07/2004, 23:59:59 Karpos PM10
08/07/2004, 00:00:00 - 04/12/2004, 23:59:59 Karpos PM10
05/12/2004, 00:00:00 - 03/05/2005, 23:59:59 Karpos PM10
04/05/2005, 00:00:00 - 30/09/2005, 23:59:59 Karpos PM10
01/10/2005, 00:00:00 - 27/02/2006, 23:59:59 Karpos PM10
28/02/2006, 00:00:00 - 27/07/2006, 23:59:59 Karpos PM10
28/07/2006, 00:00:00 - 24/12/2006, 23:59:59 Karpos PM10
25/12/2006, 00:00:00 - 23/05/2007, 23:59:59 Karp

15/02/2020, 00:00:00 - 13/07/2020, 23:59:59 Karpos O3
01/01/2000, 00:00:00 - 29/05/2000, 23:59:59 Karpos CO
30/05/2000, 00:00:00 - 26/10/2000, 23:59:59 Karpos CO
27/10/2000, 00:00:00 - 25/03/2001, 23:59:59 Karpos CO
26/03/2001, 00:00:00 - 22/08/2001, 23:59:59 Karpos CO
23/08/2001, 00:00:00 - 19/01/2002, 23:59:59 Karpos CO
20/01/2002, 00:00:00 - 18/06/2002, 23:59:59 Karpos CO
19/06/2002, 00:00:00 - 15/11/2002, 23:59:59 Karpos CO
16/11/2002, 00:00:00 - 14/04/2003, 23:59:59 Karpos CO
15/04/2003, 00:00:00 - 11/09/2003, 23:59:59 Karpos CO
12/09/2003, 00:00:00 - 08/02/2004, 23:59:59 Karpos CO
09/02/2004, 00:00:00 - 07/07/2004, 23:59:59 Karpos CO
08/07/2004, 00:00:00 - 04/12/2004, 23:59:59 Karpos CO
05/12/2004, 00:00:00 - 03/05/2005, 23:59:59 Karpos CO
04/05/2005, 00:00:00 - 30/09/2005, 23:59:59 Karpos CO
01/10/2005, 00:00:00 - 27/02/2006, 23:59:59 Karpos CO
28/02/2006, 00:00:00 - 27/07/2006, 23:59:59 Karpos CO
28/07/2006, 00:00:00 - 24/12/2006, 23:59:59 Karpos CO
25/12/2006, 00:00:00 - 23/05

15/02/2020, 00:00:00 - 13/07/2020, 23:59:59 Karpos SO2
01/01/2000, 00:00:00 - 29/05/2000, 23:59:59 Centar PM10
30/05/2000, 00:00:00 - 26/10/2000, 23:59:59 Centar PM10
27/10/2000, 00:00:00 - 25/03/2001, 23:59:59 Centar PM10
26/03/2001, 00:00:00 - 22/08/2001, 23:59:59 Centar PM10
23/08/2001, 00:00:00 - 19/01/2002, 23:59:59 Centar PM10
20/01/2002, 00:00:00 - 18/06/2002, 23:59:59 Centar PM10
19/06/2002, 00:00:00 - 15/11/2002, 23:59:59 Centar PM10
16/11/2002, 00:00:00 - 14/04/2003, 23:59:59 Centar PM10
15/04/2003, 00:00:00 - 11/09/2003, 23:59:59 Centar PM10
12/09/2003, 00:00:00 - 08/02/2004, 23:59:59 Centar PM10
09/02/2004, 00:00:00 - 07/07/2004, 23:59:59 Centar PM10
08/07/2004, 00:00:00 - 04/12/2004, 23:59:59 Centar PM10
05/12/2004, 00:00:00 - 03/05/2005, 23:59:59 Centar PM10
04/05/2005, 00:00:00 - 30/09/2005, 23:59:59 Centar PM10
01/10/2005, 00:00:00 - 27/02/2006, 23:59:59 Centar PM10
28/02/2006, 00:00:00 - 27/07/2006, 23:59:59 Centar PM10
28/07/2006, 00:00:00 - 24/12/2006, 23:59:59 Centa

21/04/2019, 00:00:00 - 17/09/2019, 23:59:59 Centar O3
18/09/2019, 00:00:00 - 14/02/2020, 23:59:59 Centar O3
15/02/2020, 00:00:00 - 13/07/2020, 23:59:59 Centar O3
01/01/2000, 00:00:00 - 29/05/2000, 23:59:59 Centar CO
30/05/2000, 00:00:00 - 26/10/2000, 23:59:59 Centar CO
27/10/2000, 00:00:00 - 25/03/2001, 23:59:59 Centar CO
26/03/2001, 00:00:00 - 22/08/2001, 23:59:59 Centar CO
23/08/2001, 00:00:00 - 19/01/2002, 23:59:59 Centar CO
20/01/2002, 00:00:00 - 18/06/2002, 23:59:59 Centar CO
19/06/2002, 00:00:00 - 15/11/2002, 23:59:59 Centar CO
16/11/2002, 00:00:00 - 14/04/2003, 23:59:59 Centar CO
15/04/2003, 00:00:00 - 11/09/2003, 23:59:59 Centar CO
12/09/2003, 00:00:00 - 08/02/2004, 23:59:59 Centar CO
09/02/2004, 00:00:00 - 07/07/2004, 23:59:59 Centar CO
08/07/2004, 00:00:00 - 04/12/2004, 23:59:59 Centar CO
05/12/2004, 00:00:00 - 03/05/2005, 23:59:59 Centar CO
04/05/2005, 00:00:00 - 30/09/2005, 23:59:59 Centar CO
01/10/2005, 00:00:00 - 27/02/2006, 23:59:59 Centar CO
28/02/2006, 00:00:00 - 27/07

21/04/2019, 00:00:00 - 17/09/2019, 23:59:59 Centar SO2
18/09/2019, 00:00:00 - 14/02/2020, 23:59:59 Centar SO2
15/02/2020, 00:00:00 - 13/07/2020, 23:59:59 Centar SO2
01/01/2000, 00:00:00 - 29/05/2000, 23:59:59 GaziBaba PM10
30/05/2000, 00:00:00 - 26/10/2000, 23:59:59 GaziBaba PM10
27/10/2000, 00:00:00 - 25/03/2001, 23:59:59 GaziBaba PM10
26/03/2001, 00:00:00 - 22/08/2001, 23:59:59 GaziBaba PM10
23/08/2001, 00:00:00 - 19/01/2002, 23:59:59 GaziBaba PM10
20/01/2002, 00:00:00 - 18/06/2002, 23:59:59 GaziBaba PM10
19/06/2002, 00:00:00 - 15/11/2002, 23:59:59 GaziBaba PM10
16/11/2002, 00:00:00 - 14/04/2003, 23:59:59 GaziBaba PM10
15/04/2003, 00:00:00 - 11/09/2003, 23:59:59 GaziBaba PM10
12/09/2003, 00:00:00 - 08/02/2004, 23:59:59 GaziBaba PM10
09/02/2004, 00:00:00 - 07/07/2004, 23:59:59 GaziBaba PM10
08/07/2004, 00:00:00 - 04/12/2004, 23:59:59 GaziBaba PM10
05/12/2004, 00:00:00 - 03/05/2005, 23:59:59 GaziBaba PM10
04/05/2005, 00:00:00 - 30/09/2005, 23:59:59 GaziBaba PM10
01/10/2005, 00:00:00 - 

...

18/12/2013, 00:00:00 - 16/05/2014, 23:59:59 Lisice SO2
17/05/2014, 00:00:00 - 13/10/2014, 23:59:59 Lisice SO2
14/10/2014, 00:00:00 - 12/03/2015, 23:59:59 Lisice SO2
13/03/2015, 00:00:00 - 09/08/2015, 23:59:59 Lisice SO2
10/08/2015, 00:00:00 - 06/01/2016, 23:59:59 Lisice SO2
07/01/2016, 00:00:00 - 04/06/2016, 23:59:59 Lisice SO2
05/06/2016, 00:00:00 - 01/11/2016, 23:59:59 Lisice SO2
02/11/2016, 00:00:00 - 31/03/2017, 23:59:59 Lisice SO2
01/04/2017, 00:00:00 - 28/08/2017, 23:59:59 Lisice SO2
29/08/2017, 00:00:00 - 25/01/2018, 23:59:59 Lisice SO2
26/01/2018, 00:00:00 - 24/06/2018, 23:59:59 Lisice SO2
25/06/2018, 00:00:00 - 21/11/2018, 23:59:59 Lisice SO2
22/11/2018, 00:00:00 - 20/04/2019, 23:59:59 Lisice SO2
21/04/2019, 00:00:00 - 17/09/2019, 23:59:59 Lisice SO2
18/09/2019, 00:00:00 - 14/02/2020, 23:59:59 Lisice SO2
15/02/2020, 00:00:00 - 13/07/2020, 23:59:59 Lisice SO2
01/01/2000, 00:00:00 - 29/05/2000, 23:59:59 Rektorat PM10
30/05/2000, 00:00:00 - 26/10/2000, 23:59:59 Rektorat PM10
27/1

02/07/2011, 00:00:00 - 28/11/2011, 23:59:59 Rektorat O3
29/11/2011, 00:00:00 - 26/04/2012, 23:59:59 Rektorat O3
27/04/2012, 00:00:00 - 23/09/2012, 23:59:59 Rektorat O3
24/09/2012, 00:00:00 - 20/02/2013, 23:59:59 Rektorat O3
21/02/2013, 00:00:00 - 20/07/2013, 23:59:59 Rektorat O3
21/07/2013, 00:00:00 - 17/12/2013, 23:59:59 Rektorat O3
18/12/2013, 00:00:00 - 16/05/2014, 23:59:59 Rektorat O3
17/05/2014, 00:00:00 - 13/10/2014, 23:59:59 Rektorat O3
14/10/2014, 00:00:00 - 12/03/2015, 23:59:59 Rektorat O3
13/03/2015, 00:00:00 - 09/08/2015, 23:59:59 Rektorat O3
10/08/2015, 00:00:00 - 06/01/2016, 23:59:59 Rektorat O3
07/01/2016, 00:00:00 - 04/06/2016, 23:59:59 Rektorat O3
05/06/2016, 00:00:00 - 01/11/2016, 23:59:59 Rektorat O3
02/11/2016, 00:00:00 - 31/03/2017, 23:59:59 Rektorat O3
01/04/2017, 00:00:00 - 28/08/2017, 23:59:59 Rektorat O3
29/08/2017, 00:00:00 - 25/01/2018, 23:59:59 Rektorat O3
26/01/2018, 00:00:00 - 24/06/2018, 23:59:59 Rektorat O3
25/06/2018, 00:00:00 - 21/11/2018, 23:59:59 Rekt

12/06/2009, 00:00:00 - 08/11/2009, 23:59:59 Rektorat SO2
09/11/2009, 00:00:00 - 07/04/2010, 23:59:59 Rektorat SO2
08/04/2010, 00:00:00 - 04/09/2010, 23:59:59 Rektorat SO2
05/09/2010, 00:00:00 - 01/02/2011, 23:59:59 Rektorat SO2
02/02/2011, 00:00:00 - 01/07/2011, 23:59:59 Rektorat SO2
02/07/2011, 00:00:00 - 28/11/2011, 23:59:59 Rektorat SO2
29/11/2011, 00:00:00 - 26/04/2012, 23:59:59 Rektorat SO2
27/04/2012, 00:00:00 - 23/09/2012, 23:59:59 Rektorat SO2
24/09/2012, 00:00:00 - 20/02/2013, 23:59:59 Rektorat SO2
21/02/2013, 00:00:00 - 20/07/2013, 23:59:59 Rektorat SO2
21/07/2013, 00:00:00 - 17/12/2013, 23:59:59 Rektorat SO2
18/12/2013, 00:00:00 - 16/05/2014, 23:59:59 Rektorat SO2
17/05/2014, 00:00:00 - 13/10/2014, 23:59:59 Rektorat SO2
14/10/2014, 00:00:00 - 12/03/2015, 23:59:59 Rektorat SO2
13/03/2015, 00:00:00 - 09/08/2015, 23:59:59 Rektorat SO2
10/08/2015, 00:00:00 - 06/01/2016, 23:59:59 Rektorat SO2
07/01/2016, 00:00:00 - 04/06/2016, 23:59:59 Rektorat SO2
05/06/2016, 00:00:00 - 01/11/20

09/02/2004, 00:00:00 - 07/07/2004, 23:59:59 Miladinovci O3
08/07/2004, 00:00:00 - 04/12/2004, 23:59:59 Miladinovci O3
05/12/2004, 00:00:00 - 03/05/2005, 23:59:59 Miladinovci O3
04/05/2005, 00:00:00 - 30/09/2005, 23:59:59 Miladinovci O3
01/10/2005, 00:00:00 - 27/02/2006, 23:59:59 Miladinovci O3
28/02/2006, 00:00:00 - 27/07/2006, 23:59:59 Miladinovci O3
28/07/2006, 00:00:00 - 24/12/2006, 23:59:59 Miladinovci O3
25/12/2006, 00:00:00 - 23/05/2007, 23:59:59 Miladinovci O3
24/05/2007, 00:00:00 - 20/10/2007, 23:59:59 Miladinovci O3
21/10/2007, 00:00:00 - 18/03/2008, 23:59:59 Miladinovci O3
19/03/2008, 00:00:00 - 15/08/2008, 23:59:59 Miladinovci O3
16/08/2008, 00:00:00 - 12/01/2009, 23:59:59 Miladinovci O3
13/01/2009, 00:00:00 - 11/06/2009, 23:59:59 Miladinovci O3
12/06/2009, 00:00:00 - 08/11/2009, 23:59:59 Miladinovci O3
09/11/2009, 00:00:00 - 07/04/2010, 23:59:59 Miladinovci O3
08/04/2010, 00:00:00 - 04/09/2010, 23:59:59 Miladinovci O3
05/09/2010, 00:00:00 - 01/02/2011, 23:59:59 Miladinovci 

15/02/2020, 00:00:00 - 13/07/2020, 23:59:59 Miladinovci NO2
01/01/2000, 00:00:00 - 29/05/2000, 23:59:59 Miladinovci SO2
30/05/2000, 00:00:00 - 26/10/2000, 23:59:59 Miladinovci SO2
27/10/2000, 00:00:00 - 25/03/2001, 23:59:59 Miladinovci SO2
26/03/2001, 00:00:00 - 22/08/2001, 23:59:59 Miladinovci SO2
23/08/2001, 00:00:00 - 19/01/2002, 23:59:59 Miladinovci SO2
20/01/2002, 00:00:00 - 18/06/2002, 23:59:59 Miladinovci SO2
19/06/2002, 00:00:00 - 15/11/2002, 23:59:59 Miladinovci SO2
16/11/2002, 00:00:00 - 14/04/2003, 23:59:59 Miladinovci SO2
15/04/2003, 00:00:00 - 11/09/2003, 23:59:59 Miladinovci SO2
12/09/2003, 00:00:00 - 08/02/2004, 23:59:59 Miladinovci SO2
09/02/2004, 00:00:00 - 07/07/2004, 23:59:59 Miladinovci SO2
08/07/2004, 00:00:00 - 04/12/2004, 23:59:59 Miladinovci SO2
05/12/2004, 00:00:00 - 03/05/2005, 23:59:59 Miladinovci SO2
04/05/2005, 00:00:00 - 30/09/2005, 23:59:59 Miladinovci SO2
01/10/2005, 00:00:00 - 27/02/2006, 23:59:59 Miladinovci SO2
28/02/2006, 00:00:00 - 27/07/2006, 23:59